In [1]:
import gdelt
import os
from tqdm import tnrange
import sys
import warnings
import pandas as pd

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
class Debugger(object):
    enabled = False
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        if self.enabled:
            print('Entering', self.func.__name__)
            print('    args:', args, kwargs)
        return self.func(*args, **kwargs)

Debugger.enabled = True

In [3]:
gd2 = gdelt.gdelt(version=2)


In [4]:
import requests
import urllib3
import time
# Full day pull, output to pandas dataframe, events table
lengths = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30 ,31]

def pull_month(month):
    month_str = str(month).zfill(2)
    for date in tnrange(1, lengths[month - 1] + 1):
        for attempt in range(5):
            try:
                date = str(date).zfill(2)
                if not os.path.exists("../data/indonesia/raw/2019{}{}.csv".format(month_str, date)):
                    print(['2019 {} {}'.format(month_str, date)])
                    results = gd2.Search(['2019 {} {}'.format(month_str, date)],
                                         table='events',coverage=True)
                    results = results[results['ActionGeo_CountryCode'] == 'ID']
                    results.to_csv("../data/indonesia/raw/2019{}{}.csv".format(month_str, date))
            except (requests.exceptions.ReadTimeout, 
                    requests.exceptions.ConnectionError,
                    urllib3.exceptions.MaxRetryError) as e:
                print(e, attempt)
                time.sleep(attempt*10)
                continue    

In [ ]:
[pull_month(x) for x in range(1, 13)]

['2019 12 16']
